In [1]:
%pip install -qU langchain langgraph langchain-openai langchain-community langchain-text-splitters chromadb pypdf


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
OPENAI_API_KEY = "<OPENAI_KEY>"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [5]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve()))

from claim_verifier import build_retriever, make_search_tool, SYSTEM_PROMPT

PDF_PATH = "data/attention_all_u_need.pdf"

retriever = build_retriever(
    pdf_path=PDF_PATH,
    persist_dir="chroma_claim_db",
    collection_name="claim_pdf",
    k=5,
)
claim_tool = make_search_tool(retriever)
print("Tool name:", claim_tool.name)


/Users/satyammittal/github/AI-ML/demo_applications/agent-patterns-pack/claim_verifier.py:57: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vs = Chroma(


Tool name: claim_verifier


In [12]:
def call_claim_tool(query: str) -> str:
    # Try common invoke patterns across versions
    try:
        return claim_tool.invoke({"query": query})
    except Exception:
        try:
            return claim_tool.invoke(query)
        except Exception:
            # Fallback: call underlying function directly (rare)
            return claim_tool.func(query)


In [13]:
from langchain_openai import ChatOpenAI
planner_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
executor_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


In [6]:
def call_claim_tool(query: str) -> str:
    # Try common invoke patterns across versions
    try:
        return claim_tool.invoke({"query": query})
    except Exception:
        try:
            return claim_tool.invoke(query)
        except Exception:
            # Fallback: call underlying function directly (rare)
            return claim_tool.func(query)


In [7]:
from langchain_openai import ChatOpenAI
planner_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
executor_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


In [8]:
def make_plan(user_question: str) -> list[str]:
    prompt = f"""
You are planning steps to answer a user question using a PDF search tool named `{claim_tool.name}`.

User question:
{user_question}

Return a plan as 3-6 bullet points, each starting with "- ".
Keep steps concrete and searchable (what to look up in the policy).
"""
    msg = planner_llm.invoke(prompt)
    steps = [ln[2:].strip() for ln in msg.content.splitlines() if ln.strip().startswith("- ")]
    if not steps:
        steps = [
            "Search the policy for the most relevant section(s)",
            "Extract the key rule(s) with page citations",
            "Write a final answer with citations"
        ]
    return steps


In [9]:
def execute_plan(user_question: str, plan: list[str]) -> dict:
    scratch = []

    for i, step in enumerate(plan, start=1):
        # Generate best search query for the step
        q_prompt = f"""
Generate a short search query to use with the tool `{claim_tool.name}`.

User question: {user_question}
Current step: {step}

Return ONLY the query text.
"""
        query_msg = executor_llm.invoke(q_prompt)
        search_query = query_msg.content.strip().strip('"')

        evidence = call_claim_tool(search_query)

        # Summarize evidence for this step (keep citations)
        s_prompt = f"""
You are extracting useful info from cited evidence.

User question: {user_question}
Step {i}: {step}

Evidence:
{evidence}

Write 2-6 bullet points of findings. Each bullet MUST include (page X) based on evidence lines.
If evidence is irrelevant, say "No useful evidence found."
"""
        summary_msg = executor_llm.invoke(s_prompt)
        scratch.append(f"## Step {i}: {step}\nQuery: {search_query}\n{summary_msg.content}")

    # Final answer
    final_prompt = f"""
{SYSTEM_PROMPT}

Tool name available: {claim_tool.name}

User question:
{user_question}

Notes from plan execution:
{chr(10).join(scratch)}

Write a FINAL answer with citations like (page X).
If not found, write exactly: Not found in the provided PDF.
"""
    final_msg = executor_llm.invoke(final_prompt)

    return {
        "plan": plan,
        "scratch": "\n\n".join(scratch),
        "final": final_msg.content.strip()
    }


In [11]:
question = "What does the transforer say about data softmax and logit?"
plan = make_plan(question)
out = execute_plan(question, plan)

print("PLAN:")
for s in out["plan"]:
    print("-", s)

print("\nFINAL:\n", out["final"])


PLAN:
- Search for the term "transformer" in the PDF to locate sections discussing its architecture and components.
- Look for keywords "data softmax" to find explanations or definitions related to softmax functions in the context of transformers.
- Search for "logit" to identify any references to logits, particularly in relation to the output layer of the transformer model.
- Check for any sections that discuss the relationship between softmax and logits, especially in terms of how they are used in the transformer’s prediction process.
- Review any examples or diagrams that illustrate the use of softmax and logits within the transformer framework for better understanding.

FINAL:
 The Transformer architecture employs a learned linear transformation followed by a softmax function to convert the decoder output into predicted next-token probabilities. This process involves using learned embeddings to transform input and output tokens into vectors of dimension \(d_{model} = 512\) (page 4)